In [1]:
import pandas as pd

In [14]:
doxyDonkeyPosts = pd.read_csv('doxyDonkeyPosts.csv')
doxyDonkeyPosts = doxyDonkeyPosts['Posts'].values.astype('U')

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [16]:
vectorizer = TfidfVectorizer(max_df=0.5, min_df=2, stop_words='english')

In [17]:
X = vectorizer.fit_transform(doxyDonkeyPosts)

In [22]:
from sklearn.cluster import KMeans

In [23]:
km = KMeans(n_clusters=3, init='k-means++', max_iter= 100, n_init=1, verbose=True)
km.fit(X)

Initialization complete
Iteration  0, inertia 5359.553
Iteration  1, inertia 2694.530
Iteration  2, inertia 2687.174
Iteration  3, inertia 2685.258
Iteration  4, inertia 2684.135
Iteration  5, inertia 2683.021
Iteration  6, inertia 2681.509
Iteration  7, inertia 2680.700
Iteration  8, inertia 2680.256
Iteration  9, inertia 2680.020
Iteration 10, inertia 2679.886
Iteration 11, inertia 2679.843
Iteration 12, inertia 2679.813
Iteration 13, inertia 2679.804
Iteration 14, inertia 2679.799
Iteration 15, inertia 2679.797
Iteration 16, inertia 2679.793
Iteration 17, inertia 2679.789
Iteration 18, inertia 2679.787
Iteration 19, inertia 2679.786
Converged at iteration 19: center shift 0.000000e+00 within tolerance 7.339674e-09


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
       n_clusters=3, n_init=1, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=True)

In [24]:
import numpy as np

In [26]:
np.unique(km.labels_, return_counts=True)

(array([0, 1, 2], dtype=int32), array([ 856, 1693,  255]))